<h1>Load Data</h1>

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

### beers and breweries

In [30]:
beer_file = r'../data/bigData/beers.csv'
breweries_file = r'../data/bigData/breweries.csv'

beers = pd.read_csv (beer_file)

breweries = pd.read_csv (breweries_file)
breweries.rename(columns={'id':'brewery_id'},inplace=True)

reviews = pd.read_csv (r"../data/bigData/reviews.csv")

### long and lat of cities

In [31]:
cities_pos_file  = r'lat_long_03-05-2020_19:21.csv' 

cities_pos = pd.read_csv (cities_pos_file)

### join_beer_brewery

In [32]:
beer_and_breweries = pd.merge(beers,breweries,on="brewery_id")
beer_and_breweries.rename(columns={'name_x':'beer','name_y':'brewery','country_y':'country'},inplace=True)
beer_and_breweries = beer_and_breweries[["id","beer","style","availability","abv","retired","brewery","city","country","types"]]


### cleaning

In [33]:
beer_and_breweries = beer_and_breweries[beer_and_breweries["abv"]<70]

Delete redondant data

In [34]:
del beers
del breweries

### print data infos

In [12]:
beer_and_breweries.head(2)

,id,beer,style,availability,abv,retired,brewery,city,country,types
0,202522,Olde Cogitator,English Oatmeal Stout,Rotating,7.3,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery"
1,8677,Red Leaf Strong Ale,American Strong Ale,Year-round,8.6,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery"


In [13]:
cities_pos.head(2)

,Country,City,lat,long
0,BE,Erpe-Mere,50.93604,4.03550
1,US,Thousand Palms,33.82002,-116.39029


In [14]:
reviews.head(2)

,beer_id,username,date,text,look,smell,taste,feel,overall,score
0,271781,bluejacket74,2017-03-17,"750 ml bottle, 2016 vintage, bottle #304 of...",4.0,4.0,4.0,4.25,4.0,4.03
1,125646,_dirty_,2017-12-21,,4.5,4.5,4.5,4.50,4.5,4.50


---
# Raw data to decent json

In [26]:
beer_and_breweries.set_index("id").to_json("../data/raw_data.json",orient='records')

KeyError: "None of ['id'] are in the columns"

In [67]:
bpc = beer_and_breweries.groupby("country").apply(lambda x: x.to_json(orient='records'))
bpc.to_json("../data/raw_data_per_c.json",orient="index")

---
# Infos per country

In [66]:
countries = beer_and_breweries[["country"]].drop_duplicates()
len(countries)

187

### nb_of_beers

In [67]:
temp = beer_and_breweries[["country","id"]].groupby("country").count()
temp.rename(columns={'id':'N_beer'},inplace=True)
countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
del temp
countries.head()

,country,N_beer
0,US,231940.0
1,NO,1095.0
2,JP,1582.0
3,IT,2523.0
4,CA,20053.0


### Average abv/beer

In [68]:
temp = beer_and_breweries[["country","abv"]].groupby("country").mean()
temp.rename(columns={'abv':'avg_abv'},inplace=True)
temp.head()

,avg_abv
country,
AD,6.188889
AE,4.690000
AG,5.200000
AL,4.700000
AM,5.087500


In [69]:
countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
del temp
countries.head()

,country,N_beer,avg_abv
0,US,231940.0,6.721202
1,NO,1095.0,6.403205
2,JP,1582.0,5.682332
3,IT,2523.0,6.400460
4,CA,20053.0,6.096917


### median abv

In [70]:
temp = beer_and_breweries[["country","abv"]].groupby("country").median()
temp.rename(columns={'abv':'median_abv'},inplace=True)
temp.head()

,median_abv
country,
AD,5.60
AE,4.69
AG,5.20
AL,4.30
AM,4.85


In [71]:
countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
del temp
countries.head()

,country,N_beer,avg_abv,median_abv
0,US,231940.0,6.721202,6.3
1,NO,1095.0,6.403205,6.0
2,JP,1582.0,5.682332,5.0
3,IT,2523.0,6.400460,6.0
4,CA,20053.0,6.096917,5.6


### most abv

In [72]:
temp = beer_and_breweries[["country","abv"]].groupby("country").max()
temp.rename(columns={'abv':'max_abv'},inplace=True)
temp.head()

,max_abv
country,
AD,8.50
AE,4.69
AG,5.60
AL,6.50
AM,8.50


In [73]:
countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
del temp
countries.head()

,country,N_beer,avg_abv,median_abv,max_abv
0,US,231940.0,6.721202,6.3,56.0
1,NO,1095.0,6.403205,6.0,20.0
2,JP,1582.0,5.682332,5.0,15.0
3,IT,2523.0,6.400460,6.0,16.0
4,CA,20053.0,6.096917,5.6,66.0


### number of breweries

In [74]:
temp = beer_and_breweries[["country","brewery"]].groupby("country").nunique()[["brewery"]]
temp.rename(columns={'brewery':'n_brewery'},inplace=True)
temp.head()

,n_brewery
country,
AD,1
AE,1
AG,1
AL,7
AM,4


In [75]:
countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
del temp
countries.head()

,country,N_beer,avg_abv,median_abv,max_abv,n_brewery
0,US,231940.0,6.721202,6.3,56.0,7338.0
1,NO,1095.0,6.403205,6.0,20.0,77.0
2,JP,1582.0,5.682332,5.0,15.0,164.0
3,IT,2523.0,6.400460,6.0,16.0,367.0
4,CA,20053.0,6.096917,5.6,66.0,940.0


---
# Draft

In [37]:
beer_and_breweries.loc[beer_and_breweries["abv"].idxmax()]

id                              104620
beer                       Snake Venom
style               British Barleywine
availability     Limited (brewed once)
abv                               67.5
retired                              t
brewery                    Brewmeister
city                            Aboyne
country                             GB
types                          Brewery
Name: 325099, dtype: object

## Beer types

In [35]:
beer_types = beer_and_breweries[['style','id']].groupby('style').agg('count').reset_index()

In [36]:
IPA = beer_and_breweries[beer_and_breweries['style'].str.contains('IPA')]

In [37]:
Lager = beer_and_breweries[(beer_and_breweries['style'].str.contains('Lager'))]

In [38]:
Ale = beer_and_breweries[(beer_and_breweries['style'].str.contains('Ale')) | (beer_and_breweries['style'].str.contains('Gose'))]

In [39]:
Stout = beer_and_breweries[(beer_and_breweries['style'].str.contains('Stout'))]

In [40]:
Pilsner = beer_and_breweries[beer_and_breweries['style'].str.contains('Pils')]

In [41]:
Porter = beer_and_breweries[beer_and_breweries['style'].str.contains('Porter')]

In [42]:
Fruit =beer_and_breweries[beer_and_breweries['style'].str.contains('Fruit')]

In [43]:
Roggenbier = beer_and_breweries[(beer_and_breweries['style'].str.contains('Roggen'))|(beer_and_breweries['style'].str.contains('Rye'))]

In [44]:
Herb = beer_and_breweries[(beer_and_breweries['style'].str.contains('Herb'))]

In [45]:
Smoke = beer_and_breweries[(beer_and_breweries['style'].str.contains('Smoke'))]

In [46]:
Barleywine=beer_and_breweries[(beer_and_breweries['style'].str.contains('Barleywine'))]

In [63]:
def get_data_filters(data,name,countries):
    #Number of beers
    temp = data[["country","id"]].groupby("country").count()
    temp.rename(columns={'id':'N_beer_'+name},inplace=True)
    countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
    
    #Average ABV
    temp = data[["country","abv"]].groupby("country").mean()
    temp.rename(columns={'abv':'avg_abv_'+name},inplace=True)
    countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
    
    # HIghest ABV
    temp = data[["country","abv"]].groupby("country").max()
    temp.rename(columns={'abv':'max_abv_'+name},inplace=True)
    countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')

    #Number of breweries
    temp = data[["country","brewery"]].groupby("country").nunique()[["brewery"]]
    temp.rename(columns={'brewery':'n_brewery_'+name},inplace=True)
    countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
    
    return countries
    

In [76]:
countries =get_data_filters(IPA,"IPA",countries)
countries =get_data_filters(Lager,'Lager',countries)
countries =get_data_filters(Ale,'Ale',countries)
countries =get_data_filters(Stout,"Stout",countries)
countries =get_data_filters(Pilsner,'Pilsner',countries)
countries =get_data_filters(Porter,'Porter',countries)
countries =get_data_filters(Fruit,'Fruit',countries)
countries =get_data_filters(Roggenbier,'Roggenbier',countries)
countries =get_data_filters(Herb,'Herb',countries)
countries =get_data_filters(Smoke,'Smoke',countries)
countries =get_data_filters(Barleywine,'Barleywine',countries)

## Saving to csv and json

In [ ]:
countries=countries.set_index("country")

countries.to_csv(r"../data/countries_basic_data.csv")
countries.to_json(r"../data/countries_basic_data.json",orient='index')

## Reviews for cloud words

In [22]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import string
from tqdm import tqdm_notebook

In [ ]:
beer_and_breweries = beer_and_breweries.set_index('id')

In [297]:
def lower_punct_removal(text):
    temp = text.lower()
    return temp.translate(str.maketrans('', '', string.punctuation))
def stopwords_removal(text):
    obvious= ['beer','nice','like','good','well','bottle','bit','head','taste','flavor','smell','little',\
              'glass','pours','notes','body','carbonation','color','hops','really','mouthfeel','aroma','malt',\
             'one','overall','would','finish','much','medium','poured']
    return ' '.join([x for x in text.split() if not (x in stopwords_en or x.isdigit() or x in obvious) ])



def get_words_counts(beer_sub_type,number):
    barley_reviews = reviews[reviews['beer_id'].isin(beer_sub_type.index.to_list())].copy()
    barley_reviews['text']  = barley_reviews['text'].apply(lower_punct_removal)
    barley_reviews['text']=barley_reviews['text'].apply(stopwords_removal)
    
    counter = CountVectorizer()
    X = counter.fit_transform(list(barley_reviews['text']))
    word_count= np.asarray(X.sum(axis =0))[0]
    
    indices = np.argsort(word_count)[-number:]
    counts = np.sort(word_count)[-number:]
    
    return np.array(counter.get_feature_names())[indices],counts


def store_words_(data,beer_types):
    result = pd.DataFrame(columns=['beer_type','words'])
    result['beer_type'] = beer_types
    for j in tqdm_notebook(range(len(beer_types))):
        words,counts = get_words_counts(data[j],20)
        liste = []
        for i in range(len(words)):
            liste.append({"x":words[i],"value": counts[i]})
        result.loc[beer_types[j],'words'] = liste

    return result

In [101]:
stopwords_en =stopwords.words('english')

In [216]:
reviews = reviews[reviews['text']!= '\xa0\xa0']

In [283]:
get_words_counts(Barleywine,20)

(array(['malts', 'sugar', 'vanilla', 'smooth', 'big', 'bourbon', 'flavors',
        'nose', 'lacing', 'sweetness', 'fruit', 'hop', 'toffee', 'light',
        'brown', 'barleywine', 'dark', 'caramel', 'alcohol', 'sweet'],
       dtype='<U81'),
 array([17560, 17806, 17983, 18226, 18891, 18897, 18947, 19745, 20149,
        20221, 20380, 21117, 21269, 26439, 31552, 33089, 41850, 49512,
        50082, 53089], dtype=int64))

In [284]:
get_words_counts(IPA,20)

(array(['alcohol', 'sweetness', 'floral', 'hoppy', 'fruit', 'caramel',
        'nose', 'great', 'lacing', 'white', 'bitter', 'grapefruit',
        'sweet', 'pine', 'light', 'bitterness', 'orange', 'citrus', 'hop',
        'ipa'], dtype='<U235'),
 array([115175, 116725, 117310, 118542, 118895, 144788, 147053, 158205,
        195802, 202436, 209962, 213962, 220079, 227207, 230014, 231322,
        239832, 338567, 343219, 355327], dtype=int64))

In [285]:
get_words_counts(Porter,20)

(array(['caramel', 'bitter', 'nose', 'great', 'flavors', 'creamy',
        'vanilla', 'smooth', 'tan', 'lacing', 'malts', 'sweet', 'black',
        'light', 'brown', 'roasted', 'porter', 'coffee', 'chocolate',
        'dark'], dtype='<U103'),
 array([ 30712,  31195,  31819,  32097,  34885,  36019,  38778,  40206,
         40596,  41623,  44958,  53890,  65132,  71082,  74439,  81359,
        100837, 118149, 131624, 134195], dtype=int64))

In [286]:
get_words_counts(Pilsner,20)

(array(['slightly', 'pale', 'bitter', 'floral', 'dry', 'yellow', 'grassy',
        'style', 'clean', 'lacing', 'bitterness', 'clear', 'pils',
        'golden', 'sweet', 'crisp', 'hop', 'white', 'pilsner', 'light'],
       dtype='<U80'),
 array([14876, 14991, 15144, 15144, 16120, 16292, 16615, 17903, 19306,
        19449, 20514, 21196, 21229, 21730, 23886, 25296, 31371, 32534,
        35235, 51075], dtype=int64))

In [287]:
get_words_counts(Fruit,20)

(array(['flavors', 'great', 'slightly', 'cherries', 'fruity', 'pretty',
        'drink', 'nose', 'sweetness', 'red', 'wheat', 'lacing', 'white',
        'raspberry', 'sour', 'cherry', 'tart', 'fruit', 'light', 'sweet'],
       dtype='<U133'),
 array([13637, 13727, 13817, 14179, 14498, 14540, 14985, 16012, 16625,
        16628, 16672, 17820, 21679, 22410, 24255, 26611, 27783, 37319,
        46716, 48155], dtype=int64))

In [288]:
get_words_counts(Smoke,20)

(array(['smokiness', 'nose', 'tan', 'smooth', 'flavors', 'smokey',
        'lacing', 'smoky', 'malts', 'coffee', 'black', 'sweet', 'roasted',
        'porter', 'brown', 'light', 'chocolate', 'dark', 'smoked', 'smoke'],
       dtype='<U108'),
 array([ 3399,  3444,  3472,  3636,  3835,  4166,  4379,  4461,  4865,
         4938,  4984,  5051,  5552,  6473,  6676,  8264,  8391, 10888,
        11397, 17502], dtype=int64))

In [289]:
get_words_counts(Stout,20)

(array(['great', 'bitter', 'flavors', 'bourbon', 'malts', 'lacing',
        'thick', 'creamy', 'smooth', 'alcohol', 'vanilla', 'brown',
        'light', 'sweet', 'roasted', 'stout', 'black', 'coffee', 'dark',
        'chocolate'], dtype='<U193'),
 array([ 97684,  98334, 103069, 107168, 111018, 111717, 114940, 117970,
        122051, 127354, 132964, 150820, 151876, 168854, 218100, 255488,
        255555, 367834, 379359, 443778], dtype=int64))

In [298]:
beer_types = ['IPA','Lager','Ale','Stout','Pilsner','Porter','Fruit','Roggenbier','Smoke','Barleywine','Herb']
data = [IPA, Lager, Ale, Stout,Pilsner,Porter,Fruit,Roggenbier,Smoke,Barleywine,Herb]

result = store_words_(data,beer_types)